In [2]:
import pandas as pd
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
import warnings as wr
wr.filterwarnings('ignore')

###  I - Pour data_2018.Json

In [4]:
with open("C:/Users/chris/Desktop/Brief_World_Cup/data_2018.json", encoding='utf-8') as f:
    data = json.load(f)

In [6]:
import pandas as pd

# Création des dictionnaires de référence

teams_dict = {equipe['id']: equipe['name'] for equipe in data['teams']}
stades_dict = {stade['id']: stade['city'] for stade in data['stadiums']}

# Liste vide pour stocker tous les matches
all_matches = []

def transform_match(donnees_match, tournoi, phase):
    """
    Transforme les données brutes d'un match en format standardisé
    """
    for match in donnees_match:
        details_match = {
            'id_match': match['name'],
            'equipe_domicile': teams_dict[match['home_team']],
            'equipe_exterieur': teams_dict[match['away_team']],
            'score_domicile': match['home_result'],
            'score_exterieur': match['away_result'],
            'date_match': match['date'],
            'phase_tournoi': phase,
            'ville': stades_dict[match['stadium']],
            'competition': tournoi
        }
        all_matches.append(details_match)

# Traitement des matches de groupe
for groupe, info_groupe in data['groups'].items():
    transform_match(info_groupe['matches'], '2018-RUSSIA', 'Phase de groupes')

# Définition des phases finales
final_stages = {
    'round_16': 'Huitièmes de finale',
    'round_8': 'Quarts de finale',
    'round_4': 'Demi-finales',
    'round_2_loser': 'Match pour la 3ème place',
    'round_2': 'Finale'
}

# Traitement des matches à élimination directe
for key_phase, name_phase in final_stages.items():
    transform_match(data['knockout'][key_phase]['matches'], '2018-RUSSIA', name_phase)

# Création du DataFrame final
df_2018 = pd.DataFrame(all_matches)

# Renommage des colonnes pour correspondre au format souhaité
final_columns = {
    'id_match': 'id_match',
    'equipe_domicile': 'home_team',
    'equipe_exterieur': 'away_team',
    'score_domicile': 'home_result',
    'score_exterieur': 'away_result',
    'date_match': 'date',
    'phase_tournoi': 'round',
    'ville': 'city',
    'competition': 'édition'
}

df_2018 = df_2018.rename(columns=final_columns)

# Réorganisation des colonnes dans l'ordre souhaité
columns_order = ['id_match', 'home_team', 'away_team', 'home_result', 'away_result', 
                 'date', 'round', 'city', 'édition']
df_2018 = df_2018[columns_order]

# Affichage du résultat
print("Aperçu du DataFrame final:")
df_2018.head()

Aperçu du DataFrame final:


,id_match,home_team,away_team,home_result,away_result,date,round,city,édition
0,1,Russia,Saudi Arabia,5,0,2018-06-14T18:00:00+03:00,Phase de groupes,Moscow,2018-RUSSIA
1,2,Egypt,Uruguay,0,1,2018-06-15T17:00:00+05:00,Phase de groupes,Yekaterinburg,2018-RUSSIA
2,17,Russia,Egypt,3,1,2018-06-19T21:00:00+03:00,Phase de groupes,Saint Petersburg,2018-RUSSIA
3,18,Uruguay,Saudi Arabia,1,0,2018-06-20T18:00:00+03:00,Phase de groupes,Rostov-on-Don,2018-RUSSIA
4,33,Uruguay,Russia,3,0,2018-06-25T18:00:00+04:00,Phase de groupes,Samara,2018-RUSSIA


In [8]:
df_2018.tail()

,id_match,home_team,away_team,home_result,away_result,date,round,city,édition
59,60,Sweden,England,0,2,2018-07-07T18:00:00+04:00,Quarts de finale,Samara,2018-RUSSIA
60,61,France,Belgium,1,0,2018-07-10T21:00:00+03:00,Demi-finales,Saint Petersburg,2018-RUSSIA
61,62,Croatia,England,2,1,2018-07-11T21:00:00+03:00,Demi-finales,Moscow,2018-RUSSIA
62,63,Belgium,England,2,0,2018-07-14T17:00:00+03:00,Match pour la 3ème place,Saint Petersburg,2018-RUSSIA
63,64,France,Croatia,4,2,2018-07-15T18:00:00+03:00,Finale,Moscow,2018-RUSSIA


In [10]:
df_2018.shape

(64, 9)

In [12]:
#df_2018.describe

In [14]:
#df_2018.info

In [16]:
#df_2018.dtypes

### II - Pour data_matches_1930_2010

In [18]:
# lecture du chemin du fichier matches.csv (1930_2010)
df_matches=pd.read_csv(r"C:\\Users\\chris\\Desktop\\Brief_World_Cup\\matches.csv", sep=',')

In [20]:
# 1. Lecture du fichier CSV des matches
file_path = "C:/Users/chris/Desktop/Brief_World_Cup/matches.csv"
data = pd.read_csv(file_path)

# 2. Création d'un identifiant unique pour chaque match
data['id_match'] = range(1, len(data) + 1)

# 3. Fonction pour extraire les scores du format "X-Y"
def extraire_scores(score_texte):
    """
    Convertit une chaîne de score (ex: "2-1") en deux nombres distincts
    Args:
        score_texte: String contenant le score (ex: "2-1")
    Returns:
        tuple: (score_domicile, score_exterieur)
    """
    try:
        # Prend uniquement la première partie avant espace (ignore les scores après prolongation)
        score_texte = score_texte.split(' ')[0]  
        # Sépare et convertit les scores en nombres
        score_domicile, score_exterieur = map(int, score_texte.split('-'))
        return score_domicile, score_exterieur
    except (ValueError, IndexError):
        # Retourne None si le format n'est pas valide
        return None, None  

# Application de l'extraction des scores
data[['score_domicile', 'score_exterieur']] = data['score'].apply(
    lambda x: pd.Series(extraire_scores(x))
)

# 4. Conversion des années en dates complètes
data['date'] = pd.to_datetime(data['year'].astype(str) + '-01-01')

# 5. Extraction des villes depuis les lieux (venues)
data['ville'] = data['venue'].str.split('.').str[0]

# 6. Sélection des colonnes pertinentes et renommage
selected_columns = [
    'id_match', 'team1', 'team2', 
    'score_domicile', 'score_exterieur', 
    'date', 'round', 'ville', 'edition'
]
df_matches = data[selected_columns].copy()

# Renommage des colonnes en français
new_name = {
    'team1': 'home_team',          
    'team2': 'away_team',
    'score_domicile': 'home_result',
    'score_exterieur': 'away_result',
    'ville': 'city',
    'edition': 'édition'
}
df_matches = df_matches.rename(columns=new_name)

# 7. Filtrage pour exclure la Coupe du Monde 2014
df_matches = df_matches[df_matches['édition'] != '2014-BRAZIL']

print("\nAperçu des données:")
#df_matches


Aperçu des données:


In [22]:
df_matches.head()

,id_match,home_team,away_team,home_result,away_result,date,round,city,édition
0,1,France,Mexico (México),4.0,1.0,1930-01-01,GROUP_STAGE,Montevideo,1930-URUGUAY
1,2,USA,Belgium (België),3.0,0.0,1930-01-01,GROUP_STAGE,Montevideo,1930-URUGUAY
2,3,Yugoslavia (Југославија),Brazil (Brasil),2.0,1.0,1930-01-01,GROUP_STAGE,Montevideo,1930-URUGUAY
3,4,Romania (România),Peru (Perú),3.0,1.0,1930-01-01,GROUP_STAGE,Montevideo,1930-URUGUAY
4,5,Argentina,France,1.0,0.0,1930-01-01,GROUP_STAGE,Montevideo,1930-URUGUAY


In [24]:
df_matches.tail()

,id_match,home_team,away_team,home_result,away_result,date,round,city,édition
6410,6411,Paraguay,Spain (España),0.0,1.0,2010-01-01,1/4_FINAL,Johannesburg,2010-SOUTH AFRICA
6411,6412,Uruguay,Netherlands (Nederland),2.0,3.0,2010-01-01,1/2_FINAL,Cape Town,2010-SOUTH AFRICA
6412,6413,Germany (Deutschland),Spain (España),0.0,1.0,2010-01-01,1/2_FINAL,Durban,2010-SOUTH AFRICA
6413,6414,Uruguay,Germany (Deutschland),2.0,3.0,2010-01-01,PLACES_3&4,Port_Elizabeth,2010-SOUTH AFRICA
6414,6415,Netherlands (Nederland),Spain (España),0.0,1.0,2010-01-01,_FINAL,Johannesburg,2010-SOUTH AFRICA


In [26]:
df_matches.shape

(6415, 9)

In [28]:
#df_matches.describe

In [30]:
#df_matches.info

In [32]:
#df_matches.dtypes

### III- Pour data_worldcup_matches_2014

In [34]:
# lecture du chemin du fichier worldcup_matches_2014.csv
df_worldcup_matches = pd.read_csv(r"C:\Users\chris\Desktop\Brief_World_Cup\worldcup_matches_2014.csv", sep=';')

In [36]:
df_worldcup_matches = pd.DataFrame({
    'id_match': range(1, len(df_worldcup_matches) + 1),  
    'home_team': df_worldcup_matches['Home Team Name'],
    'away_team': df_worldcup_matches['Away Team Name'],
    'home_result':df_worldcup_matches['Home Team Goals'],
    'away_result': df_worldcup_matches['Away Team Goals'],
    'date': df_worldcup_matches['Datetime'],  
    'round': df_worldcup_matches['Stage'],  
    'city': df_worldcup_matches['City'],
    'édition': '2014-BRAZIL'
})

In [38]:
df_worldcup_matches.head()

,id_match,home_team,away_team,home_result,away_result,date,round,city,édition
0,1,Brazil,Croatia,3,1,12 Jun 2014 - 17:00,Group A,Sao Paulo,2014-BRAZIL
1,2,Mexico,Cameroon,1,0,13 Jun 2014 - 13:00,Group A,Natal,2014-BRAZIL
2,3,Spain,Netherlands,1,5,13 Jun 2014 - 16:00,Group B,Salvador,2014-BRAZIL
3,4,Chile,Australia,3,1,13 Jun 2014 - 18:00,Group B,Cuiaba,2014-BRAZIL
4,5,Colombia,Greece,3,0,14 Jun 2014 - 13:00,Group C,Belo Horizonte,2014-BRAZIL


In [40]:
df_worldcup_matches.tail()

,id_match,home_team,away_team,home_result,away_result,date,round,city,édition
75,76,Netherlands,Costa Rica,0,0,05 Jul 2014 - 17:00,Quarter-finals,Salvador,2014-BRAZIL
76,77,Brazil,Germany,1,7,08 Jul 2014 - 17:00,Semi-finals,Belo Horizonte,2014-BRAZIL
77,78,Netherlands,Argentina,0,0,09 Jul 2014 - 17:00,Semi-finals,Sao Paulo,2014-BRAZIL
78,79,Brazil,Netherlands,0,3,12 Jul 2014 - 17:00,Play-off for third place,Brasilia,2014-BRAZIL
79,80,Germany,Argentina,1,0,13 Jul 2014 - 16:00,Final,Rio De Janeiro,2014-BRAZIL


In [42]:
df_worldcup_matches.shape

(80, 9)

In [44]:
#df_worldcup_matches.dtypes

In [46]:
#df_worldcup_matches.describe

### IV- Pour data_2022

In [48]:
import pandas as pd

# Lecture du fichier avec chemin raw string
file_path = r"C:\Users\chris\Desktop\Brief_World_Cup\Fifa_world_cup_matches.csv"
df = pd.read_csv(file_path)

# Création du nouveau DataFrame avec la structure souhaitée
df_2022 = pd.DataFrame()

# Création des colonnes dans l'ordre logique
columns_mapping = {
    'id_match': range(1, len(df) + 1),
    'home_team': df['team1'],
    'away_team': df['team2'],
    'home_result': df['number of goals team1'],
    'away_result': df['number of goals team2'],
    'date': df['date'],
    'round': df['category'],
    'city': 'Inconnue',
    'édition': '2022-QATAR'
}

# Remplissage du DataFrame avec toutes les colonnes
for col_name, col_data in columns_mapping.items():
    df_2022[col_name] = col_data

# Conversion de la date en format datetime si nécessaire
df_2022['date'] = pd.to_datetime(df_2022['date'])

# Affichage de validation
print("Dimensions du DataFrame:", df_2022.shape)
print("\nAperçu du DataFrame final:")
df_2022.head()

df_2022

Dimensions du DataFrame: (64, 9)

Aperçu du DataFrame final:


,id_match,home_team,away_team,home_result,away_result,date,round,city,édition
0,1,QATAR,ECUADOR,0,2,2022-11-20,Group A,Inconnue,2022-QATAR
1,2,ENGLAND,IRAN,6,2,2022-11-21,Group B,Inconnue,2022-QATAR
2,3,SENEGAL,NETHERLANDS,0,2,2022-11-21,Group A,Inconnue,2022-QATAR
3,4,UNITED STATES,WALES,1,1,2022-11-21,Group B,Inconnue,2022-QATAR
4,5,ARGENTINA,SAUDI ARABIA,1,2,2022-11-22,Group C,Inconnue,2022-QATAR
...,...,...,...,...,...,...,...,...,...
59,60,ENGLAND,FRANCE,1,2,2022-12-10,Quarter-final,Inconnue,2022-QATAR
60,61,ARGENTINA,CROATIA,3,0,2022-12-13,Semi-final,Inconnue,2022-QATAR
61,62,FRANCE,MOROCCO,2,0,2022-12-14,Semi-final,Inconnue,2022-QATAR
62,63,CROATIA,MOROCCO,2,1,2022-12-17,Play-off for third place,Inconnue,2022-QATAR


### V - DataFrame final après une concaténation de l'ensemble des DataFrames

In [50]:
def combine_dataframes_with_info(*dfs, df_names=None):
    # Nommage des DataFrames si non spécifié
    if df_names is None:
        df_names = [f"DataFrame_{i+1}" for i in range(len(dfs))]
    
    # Affichage des informations sur les colonnes
    print("Colonnes présentes dans chaque DataFrame :")
    for df, name in zip(dfs, df_names):
        print(f"\n{name}:")
        print(df.columns.tolist())
    
    # Combiner les DataFrames
    combined = pd.concat(dfs, ignore_index=True)
    combined['id_match'] = np.arange(1, len(combined) + 1)
    
    print("\nColonnes dans le DataFrame final :")
    print(combined.columns.tolist())
    
    return combined

# Utilisation
combined_df = combine_dataframes_with_info(
    df_matches, df_worldcup_matches, df_2018, df_2022,
    df_names=['Matches', 'WorldCup_Matches', 'Matches_2018', 'Matches_2022']
)
combined_df

Colonnes présentes dans chaque DataFrame :

Matches:
['id_match', 'home_team', 'away_team', 'home_result', 'away_result', 'date', 'round', 'city', 'édition']

WorldCup_Matches:
['id_match', 'home_team', 'away_team', 'home_result', 'away_result', 'date', 'round', 'city', 'édition']

Matches_2018:
['id_match', 'home_team', 'away_team', 'home_result', 'away_result', 'date', 'round', 'city', 'édition']

Matches_2022:
['id_match', 'home_team', 'away_team', 'home_result', 'away_result', 'date', 'round', 'city', 'édition']

Colonnes dans le DataFrame final :
['id_match', 'home_team', 'away_team', 'home_result', 'away_result', 'date', 'round', 'city', 'édition']


,id_match,home_team,away_team,home_result,away_result,date,round,city,édition
0,1,France,Mexico (México),4.0,1.0,1930-01-01 00:00:00,GROUP_STAGE,Montevideo,1930-URUGUAY
1,2,USA,Belgium (België),3.0,0.0,1930-01-01 00:00:00,GROUP_STAGE,Montevideo,1930-URUGUAY
2,3,Yugoslavia (Југославија),Brazil (Brasil),2.0,1.0,1930-01-01 00:00:00,GROUP_STAGE,Montevideo,1930-URUGUAY
3,4,Romania (România),Peru (Perú),3.0,1.0,1930-01-01 00:00:00,GROUP_STAGE,Montevideo,1930-URUGUAY
4,5,Argentina,France,1.0,0.0,1930-01-01 00:00:00,GROUP_STAGE,Montevideo,1930-URUGUAY
...,...,...,...,...,...,...,...,...,...
6618,6619,ENGLAND,FRANCE,1.0,2.0,2022-12-10 00:00:00,Quarter-final,Inconnue,2022-QATAR
6619,6620,ARGENTINA,CROATIA,3.0,0.0,2022-12-13 00:00:00,Semi-final,Inconnue,2022-QATAR
6620,6621,FRANCE,MOROCCO,2.0,0.0,2022-12-14 00:00:00,Semi-final,Inconnue,2022-QATAR
6621,6622,CROATIA,MOROCCO,2.0,1.0,2022-12-17 00:00:00,Play-off for third place,Inconnue,2022-QATAR
